In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
#import calmap
import folium
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

<H1 align='center'>Coronavirus disease 2019</H1>


<p>*Coronavirus disease 2019 (COVID-19)* is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), a virus closely related to the SARS virus. The disease was discovered and named during the 2019–20 coronavirus outbreak. Those affected may develop a fever, dry cough, fatigue, and shortness of breath. A sore throat, runny nose or sneezing is less common. While the majority of cases result in mild symptoms, some can progress to pneumonia and multi-organ failure.</p>

<p>The infection is spread from one person to others via respiratory droplets produced from the airways, often during coughing or sneezing. Time from exposure to onset of symptoms is generally between 2 and 14 days, with an average of 5 days. The standard method of diagnosis is by reverse transcription polymerase chain reaction (rRT-PCR) from a nasopharyngeal swab or sputum sample, with results within a few hours to 2 days. Antibody assays can also be used, using a blood serum sample, with results within a few days. The infection can also be diagnosed from a combination of symptoms, risk factors and a chest CT scan showing features of pneumonia.</p>

<p>Correct handwashing technique, maintaining distance from people who are coughing and not touching one's face with unwashed hands are measures recommended to prevent the disease. It is also recommended to cover one's nose and mouth with a tissue or a bent elbow when coughing. Masks are recommended in those who suspect they have the virus and for those who are taking care of someone with a suspected infection but not for the general public. There is no vaccine or specific antiviral treatment, with management involving treatment of symptoms, supportive care and experimental measures.The case fatality rate is estimated at between 1% and 3%.</p>

<p>The World Health Organization (WHO) has declared the 2019–20 coronavirus outbreak a Public Health Emergency of International Concern (PHEIC). As of 7 March 2020, evidence of local transmission of the disease has been found in multiple countries across all six WHO regions.</p>

![](http://cms.qz.com/wp-content/uploads/2020/03/novel-coronavirus-covid-19.jpg?quality=75&strip=all&w=1600&h=900&crop=1)

The disease is caused by the virus severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), previously referred to as the 2019 novel coronavirus (2019-nCoV). It is primarily spread between people via respiratory droplets from coughs and sneezes.

The virus accesses host cells via angiotensin-converting enzyme 2 (ACE2). ACE2 is found in various organs of the body, but it is most abundant in the type II alveolar cells of the lungs. This explains why lungs are the most affected organs. The density of ACE2 in each patient is a determent of the severity of the disease. As the disease at the alveolare progresses, severe acute respiratory syndrome (SARS) develops, which might lead to respiratory failure and death might ensue. ACE2 might also be the path for the virus to assault he heart causing acute cardiac injury. Patients with existing cardiovascular conditions have worst prognosis.

The virus is thought to have an animal origin. Τhere has been a "continuous common source" of the outbreak in December 2019, which would imply that several animal-to-human zoonotic events occurred at the Huanan Seafood Wholesale Market. The primary source of infection became human-to-human transmission in early January 2020.

<h2>Symptoms</h2>

Reported illnesses have ranged from mild symptoms to severe illness and death for confirmed coronavirus disease 2019 (COVID-19) cases.

The following symptoms may appear 2-14 days after exposure.

* Fever  
![Fever](https://www.cdc.gov/coronavirus/2019-ncov/images/symptoms-fever.jpg)
* Cough
![Cough](https://www.cdc.gov/coronavirus/2019-ncov/images/symptoms-cough.jpg)
* Shortness of breath
![Shortness of breath](https://www.cdc.gov/coronavirus/2019-ncov/images/symptoms-shortness-breath.jpg)

In [ ]:
data = pd.read_csv("../input/corona-virus-report/covid_19_clean_complete.csv",parse_dates=['Date'])

In [ ]:
# cases 
cases = ['Confirmed', 'Deaths', 'Recovered', 'Still Infected']

# still infected = confirmed - deaths - recovered
data['Still Infected'] = data['Confirmed'] - data['Deaths'] - data['Recovered']

# replacing Mainland china with just China
data['Country/Region'] = data['Country/Region'].replace('Mainland China', 'China')

# filling missing values 
data[['Province/State']] = data[['Province/State']].fillna('NA')
data[cases] = data[cases].fillna(0)

<H2>**Note: This analysis based on the data available until 11/04/2020**</H2>

**Latest data point**

In [ ]:
temp = data.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 'Still Infected'].sum().reset_index()
temp = temp[temp['Date']==max(temp['Date'])].reset_index(drop=True)
temp

**Recovered, Still Infected, Death**

In [ ]:
tm = temp.melt(id_vars="Date", value_vars=['Still Infected', 'Deaths', 'Recovered'])
fig = px.treemap(tm, path=["variable"], values="value", height=200)
fig.show()

In [ ]:
d = data.groupby(['Province/State','Country/Region'],as_index=False)['Province/State','Country/Region','Confirmed','Recovered','Deaths'].sum()

**Confirmed cases according to Country and Region**

In [ ]:
d["world"] = "world"
fig = px.treemap(d, path=['world', 'Country/Region', 'Province/State'], values='Confirmed',
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(d['Deaths'], weights=d['Confirmed']))
fig.show()

**Deaths cases according to Country and Region**

In [ ]:
d["world"] = "world"
fig = px.treemap(d, path=['world', 'Country/Region', 'Province/State'], values='Deaths',
                  color_continuous_scale='RdBu',)
fig.show()

**Recovered percentage according to Country and Region**

In [ ]:
d['recover%'] = d['Recovered']/d['Confirmed']
d["world"] = "world"
fig = px.treemap(d, path=['world', 'Country/Region', 'Province/State'], values='recover%',
                  color_continuous_scale='RdBu',)
fig.show()

In [ ]:
del d
del temp

In [ ]:
# cases in the ships
ship = data[data['Province/State'].str.lower().str.contains('ship')]

# china and the row
china = data[data['Country/Region']=='China']
row = data[data['Country/Region']!='China']

# latest
data_latest = data[data['Date'] == max(data['Date'])].reset_index()
china_latest = data_latest[data_latest['Country/Region']=='China']
row_latest = data_latest[data_latest['Country/Region']!='China']

data_latestt_grouped_grouped = data_latest.groupby('Country/Region')['Confirmed', 'Deaths', 'Recovered', 'Still Infected'].sum().reset_index()
china_latest_grouped = china_latest.groupby('Province/State')['Confirmed', 'Deaths', 'Recovered', 'Still Infected'].sum().reset_index()
row_latest_grouped = row_latest.groupby('Country/Region')['Confirmed', 'Deaths', 'Recovered', 'Still Infected'].sum().reset_index()

**Cases over time**

In [ ]:
temp = data.groupby('Date')['Still Infected', 'Deaths', 'Recovered'].sum().reset_index()
temp = temp.melt(id_vars="Date", value_vars=['Still Infected', 'Deaths', 'Recovered'],
                 var_name='Case', value_name='Count')
temp.head()

fig = px.bar(temp, x="Date", y="Count", color='Case',
             title='Cases over time')
fig.show()


Many people are getting recovered from infections. 

**No. of cases each day**

In [ ]:
temp = data.groupby(['Country/Region', 'Date'])['Confirmed', 'Deaths', 'Recovered'].sum()
temp = temp.reset_index()
# temp.head()

fig = px.bar(temp, x="Date", y="Confirmed", color='Country/Region', orientation='v', height=600,
             title='Confirmed', color_discrete_sequence = px.colors.cyclical.mygbm)
fig.show()

fig = px.bar(temp, x="Date", y="Deaths", color='Country/Region', orientation='v', height=600,
             title='Deaths', color_discrete_sequence = px.colors.cyclical.mygbm)
fig.show()

As you can see the situation of China is going to stable and new cases are found in other countries. Especially Italy and Iran new cases and deaths of infected persons are increased. 

In [ ]:
temp_f = data_latestt_grouped_grouped.sort_values(by='Confirmed', ascending=False)
temp_f = temp_f.reset_index(drop=True)
temp_f.style.background_gradient(cmap='YlOrRd')

Chian, Italy, Iran, Korea, France, Spain, US, Switzerland and Germany Confirmed cases reached over a thousand. Outside china new cases are poping everyday.

**Countries with deaths reported**

In [ ]:
temp_flg = data_latestt_grouped_grouped[['Country/Region', 'Deaths']]
temp_flg = temp_flg.sort_values(by='Deaths', ascending=False)
temp_flg = temp_flg.reset_index(drop=True)
temp_flg = temp_flg[temp_flg['Deaths']>0]
temp_flg.style.background_gradient(cmap='Reds')

**Countries with no cases recovered**

In [ ]:
temp = data_latestt_grouped_grouped[data_latestt_grouped_grouped['Recovered']==0]
temp = temp[['Country/Region', 'Confirmed', 'Deaths', 'Recovered']]
temp = temp.sort_values('Confirmed', ascending=False)
temp = temp.reset_index(drop=True)
temp.style.background_gradient(cmap='YlOrRd')

**Countries with all cases died**

In [ ]:
temp = row_latest_grouped[row_latest_grouped['Confirmed']==
                          row_latest_grouped['Deaths']]
temp = temp[['Country/Region', 'Confirmed', 'Deaths']]
temp = temp.sort_values('Confirmed', ascending=False)
temp = temp.reset_index(drop=True)
temp.style.background_gradient(cmap='Reds')

**Countries with all the cases recovered**

In [ ]:
temp = row_latest_grouped[row_latest_grouped['Confirmed']==
                          row_latest_grouped['Recovered']]
temp = temp[['Country/Region', 'Confirmed', 'Recovered']]
temp = temp.sort_values('Confirmed', ascending=False)
temp = temp.reset_index(drop=True)
temp.style.background_gradient(cmap='Greens')

**Countries with no affected case anymore**

In [ ]:
temp = row_latest_grouped[row_latest_grouped['Confirmed']==
                          row_latest_grouped['Deaths']+
                          row_latest_grouped['Recovered']]
temp = temp[['Country/Region', 'Confirmed', 'Deaths', 'Recovered']]
temp = temp.sort_values('Confirmed', ascending=False)
temp = temp.reset_index(drop=True)
temp.style.background_gradient(cmap='Greens')

<H2>Maps</H2>

In [ ]:
# World wide

m = folium.Map(location=[0, 0], tiles='cartodbpositron',
               min_zoom=1, max_zoom=4, zoom_start=1)

for i in range(0, len(data_latest)):
    folium.Circle(
        location=[data_latest.iloc[i]['Lat'], data_latest.iloc[i]['Long']],
        color='crimson', 
        tooltip =   '<li><bold>Country : '+str(data_latest.iloc[i]['Country/Region'])+
                    '<li><bold>Province : '+str(data_latest.iloc[i]['Province/State'])+
                    '<li><bold>Confirmed : '+str(data_latest.iloc[i]['Confirmed'])+
                    '<li><bold>Deaths : '+str(data_latest.iloc[i]['Deaths'])+
                    '<li><bold>Recovered : '+str(data_latest.iloc[i]['Recovered']),
        radius=int(data_latest.iloc[i]['Confirmed'])**1.1).add_to(m)
m

**Affected Countries**

In [ ]:
fig = px.choropleth(data_latestt_grouped_grouped, locations="Country/Region", 
                    locationmode='country names', color="Confirmed", 
                    hover_name="Country/Region", range_color=[1,7000], 
                    color_continuous_scale="agsunset", 
                    title='Countries with Confirmed Cases')
fig.update(layout_coloraxis_showscale=False)
fig.show()

# ------------------------------------------------------------------------

fig = px.choropleth(data_latestt_grouped_grouped[data_latestt_grouped_grouped['Deaths']>0], 
                    locations="Country/Region", locationmode='country names',
                    color="Deaths", hover_name="Country/Region", 
                    range_color=[1,50], color_continuous_scale="agsunset",
                    title='Countries with Deaths Reported')
fig.update(layout_coloraxis_showscale=False)
fig.show()

**No. cases in each countries**

In [ ]:
fig = px.bar(data_latestt_grouped_grouped[['Country/Region', 'Confirmed']].sort_values('Confirmed', ascending=False), 
             x="Confirmed", y="Country/Region", color='Country/Region', orientation='h',
             log_x=True, color_discrete_sequence = px.colors.qualitative.Bold, title='Confirmed Cases', height=1200)
fig.show()

temp = data_latestt_grouped_grouped[['Country/Region', 'Deaths']].sort_values('Deaths', ascending=False)
fig = px.bar(temp[temp['Deaths']>0], 
             x="Deaths", y="Country/Region", color='Country/Region', title='Deaths', orientation='h',
             log_x=True, color_discrete_sequence = px.colors.qualitative.Bold)
fig.show()

**No. of new cases everyday**

In [ ]:
# In China
temp = china.groupby('Date')['Confirmed', 'Deaths', 'Recovered'].sum().diff()
temp = temp.reset_index()
temp = temp.melt(id_vars="Date", 
                 value_vars=['Confirmed', 'Deaths', 'Recovered'])

fig = px.line(temp, x="Date", y="value", color='variable', 
             title='In China')
fig.update_layout(barmode='group')
fig.show()

#-----------------------------------------------------------------------------

# ROW
temp = row.groupby('Date')['Confirmed', 'Deaths', 'Recovered'].sum().diff()
temp = temp.reset_index()
temp = temp.melt(id_vars="Date", 
                 value_vars=['Confirmed', 'Deaths', 'Recovered'])

fig = px.line(temp, x="Date", y="value", color='variable', 
             title='Outside China')
fig.update_layout(barmode='group')
fig.show()

As you can see from the above graph the situation in China is cooling down. Other Countries are getting affected at a rapid pace. Outside china new cases are found exponentially.

In [ ]:
def from_china_or_not(row):
    if row['Country/Region']=='China':
        return 'From China'
    else:
        return 'Outside China'
    
temp = data.copy()
temp['Region'] = temp.apply(from_china_or_not, axis=1)
temp = temp.groupby(['Region', 'Date'])['Confirmed', 'Deaths', 'Recovered']
temp = temp.sum().diff().reset_index()
mask = temp['Region'] != temp['Region'].shift(1)
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan

fig = px.bar(temp, x='Date', y='Confirmed', color='Region', barmode='group', 
             text='Confirmed', title='Confirmed')
fig.update_traces(textposition='outside')
fig.show()

fig = px.bar(temp, x='Date', y='Deaths', color='Region', barmode='group', 
             text='Confirmed', title='Deaths')
fig.update_traces(textposition='outside')
fig.update_traces(textangle=-90)
fig.show()

****

In [ ]:
temp = data.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']
temp = temp.sum().diff().reset_index()

mask = temp['Country/Region'] != temp['Country/Region'].shift(1)

temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan

fig = px.bar(temp, x="Date", y="Confirmed", color='Country/Region',
             title='Number of new cases everyday')
fig.show()

In [ ]:
fig = px.bar(temp[temp['Country/Region']!='China'], x="Date", y="Confirmed", color='Country/Region',
             title='Number of new cases outside China everyday')
fig.show()

In [ ]:
fig = px.bar(temp, x="Date", y="Deaths", color='Country/Region',
             title='Number of new death case reported outside China everyday')
fig.show()

In [ ]:
fig = px.bar(temp[temp['Country/Region']!='China'], x="Date", y="Deaths", color='Country/Region',
             title='Number of new death case reported outside China everyday')
fig.show()

**No. of places to which COVID-19 spread**

In [ ]:
c_spread = china[china['Confirmed']!=0].groupby('Date')['Province/State'].unique().apply(len)
c_spread = pd.DataFrame(c_spread).reset_index()

fig = px.line(c_spread, x='Date', y='Province/State', 
              title='Number of Provinces/States/Regions of China to which COVID-19 spread over the time')
fig.show()

There are no new cases reported in new places in China for a couple of days.

In [ ]:
spread = data[data['Confirmed']!=0].groupby('Date')['Country/Region'].unique().apply(len)
spread = pd.DataFrame(spread).reset_index()

fig = px.line(spread, x='Date', y='Country/Region', 
              title='Number of Countries/Regions to which COVID-19 spread over the time')
fig.show()

New places are getting affected by the virus at an alarming rate.

<h2>Steps to Prevent Illness</h2>
<p>There is currently no vaccine to prevent coronavirus disease 2019 (COVID-19).**The best way to prevent illness is to avoid being exposed to this virus.**</p>

<p>The virus is thought to spread mainly from person-to-person.</p>

* Between people who are in close contact with one another (within about 6 feet).
* Through respiratory droplets produced when an infected person coughs or sneezes.
* These droplets can land in the mouths or noses of people who are nearby or possibly be inhaled into the lungs.

<h3>Take steps to protect yourself</h3>
![](https://www.cdc.gov/coronavirus/2019-ncov/images/protect-wash-hands.png)
**Clean your hands often**
* Wash your hands often with soap and water for at least 20 seconds especially after you have been in a public place, or after blowing your nose, coughing, or sneezing.
* If soap and water are not readily available, use a hand sanitizer that contains at least 60% alcohol. Cover all surfaces of your hands and rub them together until they feel dry.
* Avoid touching your eyes, nose, and mouth with unwashed hands.


**Avoid close contact**
![](https://www.cdc.gov/coronavirus/2019-ncov/images/protect-quarantine.png)
* Avoid close contact with people who are sick
* Put distance between yourself and other people if COVID-19 is spreading in your community. This is especially important for people who are at higher risk of getting very sick.

<h4>Take steps to protect others</h4>

**Stay home if you’re sick**
![](https://www.cdc.gov/coronavirus/2019-ncov/images/COVIDweb_02_bed.png)
* Stay home if you are sick, except to get medical care. Learn what to do if you are sick.

**Cover coughs and sneezes**
![](https://www.cdc.gov/coronavirus/2019-ncov/images/COVIDweb_06_coverCough.png)
* Cover your mouth and nose with a tissue when you cough or sneeze or use the inside of your elbow.
* Throw used tissues in the trash.
* Immediately wash your hands with soap and water for at least 20 seconds. If soap and water are not readily available, clean your hands with a hand sanitizer that contains at least 60% alcohol.

**Wear a facemask if you are sick**
![](https://www.cdc.gov/coronavirus/2019-ncov/images/COVIDweb_05_mask.png)
* If you are sick:  You should wear a facemask when you are around other people (e.g., sharing a room or vehicle) and before you enter a healthcare provider’s office. If you are not able to wear a facemask (for example, because it causes trouble breathing), then you should do your best to cover your coughs and sneezes, and people who are caring for you should wear a facemask if they enter your room. Learn what to do if you are sick.
* If you are NOT sick: You do not need to wear a facemask unless you are caring for someone who is sick (and they are not able to wear a facemask). Facemasks may be in short supply and they should be saved for caregivers.

**Clean and disinfect**
![](https://www.cdc.gov/coronavirus/2019-ncov/images/COVIDweb_09_clean.png)
* Clean AND disinfect frequently touched surfaces daily. This includes tables, doorknobs, light switches, countertops, handles, desks, phones, keyboards, toilets, faucets, and sinks.
* If surfaces are dirty, clean them: Use detergent or soap and water prior to disinfection.



<h2>Thanks for Reading</h2>